# AI Cyoda configurations Q&A with RAG Langchain

This is a playground for experimenting with trino queries generation

Install requirements

In [ ]:
pip install -r ../requirements.txt

### Load environment variables

In [ ]:
from dotenv import load_dotenv
import os
import base64

load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
WORK_DIR = os.environ["WORK_DIR"]
decoded_bytes_user = base64.b64decode(os.environ["TRINO_USER"])
TRINO_USER = decoded_bytes_user.decode("utf-8")
decoded_bytes_pwd = base64.b64decode(os.environ["TRINO_PASSWORD"])
TRINO_PASSWORD = decoded_bytes_pwd.decode("utf-8")
TRINO_CONNECTION_PATH=os.environ["TRINO_CONNECTION_PATH"]
TRINO_CONNECTION_STRING = f"trino://{TRINO_USER}:{TRINO_PASSWORD}@{TRINO_CONNECTION_PATH}"

In [ ]:
%%script echo skipping
##for google colab (optional)
# This cell is optional and can be skipped
from google.colab import userdata
API_KEY = userdata.get('OPENAI_API_KEY')
WORK_DIR = userdata.get('WORK_DIR')

### Handle unsupported version of sqlite3 (optional)

In [ ]:
pip install pysqlite3-binary

In [ ]:
import sys

__import__("pysqlite3")
sys.modules["sqlite3"] = sys.modules["pysqlite3"]

### Initialize ChatOpenAI

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import GitLoader, DirectoryLoader, TextLoader
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.schema import HumanMessage

In [ ]:
llm = ChatOpenAI(
    temperature=0.7,
    max_tokens=8000,
    model="gpt-4o-mini",
    openai_api_key=OPENAI_API_KEY,
)

### Load instructions and entities from the official cyoda repository

In [ ]:
%%script echo skipping
loader = GitLoader(
    repo_path=WORK_DIR,
    branch="cyoda-ai-configurations-3.0.x",
    file_filter=lambda file_path: file_path.startswith(f"{WORK_DIR}/data/code/"),
)
docs = loader.load()
print(f"Number of documents loaded: {len(docs)}")

In [ ]:
loader = DirectoryLoader(
    f"{WORK_DIR}/data/rag/v1/trino", loader_cls=TextLoader
)
docs = loader.load()
print(f"Number of documents loaded: {len(docs)}")

### Split documents and create vectorstore

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
            documents=splits, embedding=OpenAIEmbeddings()
        )
retriever = vectorstore.as_retriever(
            search_kwargs={"k": 10}
        )

In [ ]:
count = vectorstore._collection.count()
print(count)

### Define prompts for contextualizing question and answering question

In [ ]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [ ]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

### Answer question

In [ ]:
qa_system_prompt = """You are a Trino expert. Given an input question, 
first create a syntactically correct Trino sql query to run, 
then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, 
query for at most 100 results using the LIMIT clause as per sql.
You can order the results to return the most informative data in the database.
You must query only the columns that are needed to answer the question. 
Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today\
You can do joins only on indexes!
{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

### Create retrieval chain

In [ ]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### Initialize chat history and relevant functions

In [ ]:
chat_history = {}

In [ ]:
# Function to add a message to the chat history
def add_to_chat_history(id, question, message):
    if id in chat_history:
        chat_history[id].extend([HumanMessage(content=question), message])
    else:
        chat_history[id] = [HumanMessage(content=question), message]

In [ ]:
# Function to clear chat history
def clear_chat_history(id):
    if id in chat_history:
        del chat_history[id]

In [ ]:
def ask_question(id, question):
    ai_msg = rag_chain.invoke(
        {"input": question, "chat_history": chat_history.get(id, [])}
    )
    add_to_chat_history(id, question, ai_msg["answer"])
    return ai_msg["answer"]

### Start a chat session

In [ ]:
import uuid

# Generate a unique ID for the chat session
id = uuid.uuid1()

In [ ]:
#clear chat history if necessary
clear_chat_history(id)

In [ ]:
pip install trino==0.329.0

In [ ]:
connection_string = f"trino://{TRINO_USER}:{TRINO_PASSWORD}@{TRINO_CONNECTION_PATH}/information_schema"

In [ ]:
print(connection_string)

In [ ]:
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri(connection_string)

In [ ]:
import uuid
from typing import Optional 
from langchain.agents import tool

@tool
def run_sql_query(sql_query: str) -> str:
    """Runs sql query and returns the dataset from the database."""
    try:
        result = db.run(sql_query)
        return result
    except Exception as e:
        return str(e.__cause__)

run_sql_query.invoke("SELECT schema_data_ds_id FROM sch1.ingest_data")

In [ ]:
query = "SELECT institution, COUNT(*) AS number_of_laureates \
FROM prizes.prizes_data_prizes_laureates \
GROUP BY institution \
ORDER BY number_of_laureates DESC"

schema_query = "SELECT institution, COUNT(*) AS number_of_laureates \
FROM prizes.prizes_data_prizes_laureates \
GROUP BY institution \
ORDER BY number_of_laureates DESC"
try:
    result = run_sql_query.invoke(query)
    print(result)
except Exception as e:
    print(e.__cause__)


In [ ]:
@tool
def generate_trino_sql(question: str, chat_id: str) -> str:
    """Generates sql query based on user question"""
    try:
        question = f"{question}. Remove any ; (semi colon) at the end"
        ai_msg = rag_chain.invoke(
            {"input": question, "chat_history": chat_history.get(chat_id, [])}
        )
        add_to_chat_history(id, question, ai_msg["answer"])
        return ai_msg["answer"]
    except Exception as e:
        return str(e.__cause__)

In [ ]:
tools = [run_sql_query, generate_trino_sql]

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the generate_trino_sql tool to formulate the query. As it is trino and cyoda specific. Make sure the is no semi-colon at the end of the query. Retry up to 1 time if necessary.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)


In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, vectorstore=vectorstore)


In [ ]:
result = agent_executor.invoke({"input": "What tables do you have in schema prizes? Analyse tables structure.  PLease use chat_id \"123\""})
ask_question("")

In [ ]:
print(result['output'])
ask_question("123", f"Remeber the results of sql execution: {result['output']}" )

In [ ]:
query = "SELECT institution, COUNT(*) AS number_of_laureates \
FROM prizes.prizes_data_prizes_laureates \
GROUP BY institution \
ORDER BY number_of_laureates DESC;"
result = agent_executor.invoke({"input": f"Execute {query}. If you get error, fix the query, explain how you fixesd it and retry after correcting the query. Max retries = 3"})

In [ ]:
query = "How many laureates were in 2020. Please use chat_id = 123"
result = agent_executor.invoke({"input": f"Execute {query}. If you get error, fix the query, explain how you fixesd it and retry after correcting the query. Max retries = 3"})

In [ ]:
ask_question("123", f"Remeber the results of sql execution: {result['output']}" )
print(result['output'])
ask_question("123", "How many laureates were in 2020. Please use chat_id = 123" )

In [ ]:
query = "SELECT firstname, surname, COUNT(DISTINCT category) AS number_of_categories \
FROM prizes.prizes_data_prizes_laureates \
GROUP BY firstname, surname \
HAVING COUNT(DISTINCT category) > 1"
result = agent_executor.invoke({"input": f"Execute {query}. If you get error, fix the query, explain how you fixesd it and retry after correcting the query. Max retries = 3"})

In [ ]:
print(result['output'])